# Check the mining area tiles

In [ ]:
import geopandas as gpd
import leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
import json
from tqdm import tqdm
tqdm.pandas()
import os

os.chdir("..")
root = os.path.dirname(os.getcwd())
print(root)

import sys
sys.path.append("..")
from src.utils import calculate_dimensions_km

In [ ]:
tiles_path = root + "/data/interim/tiles.gpkg"

# list the layers of the gpkg file
print(gpd.list_layers(tiles_path))
# Load the data
data = gpd.read_file(tiles_path, layer="mining_areas_square")

data.head()

In [ ]:
calculate_dimensions_km(data.geometry[0])

# iterate over df and calculate the dimensions
data["dimensions"] = data.geometry.progress_apply(calculate_dimensions_km)

In [ ]:
# add index as a column
data["index"] = data.index
data.head()

In [ ]:
# calculate overlap percentage of each tile
data["overlap_percentage"] = data["geometry"].progress_apply(lambda x: data["geometry"].apply(lambda y: x.intersection(y).area/x.area).sum() - 1)
data["overlap_percentage"].hist(bins=50)

In [ ]:
# Create a Map
m = leafmap.Map(zoom=2)

# Add the GeoDataFrame to the map
m.add_gdf(data, layer_name="tiles")

# Display the map
m

## Calculate number of groups of connected tiles

In [ ]:
import geopandas as gpd
import networkx as nx
from tqdm import tqdm

# Step 1: Create a graph
G = nx.Graph()

# Step 2: Add nodes
for idx, geom in tqdm(data.iterrows(), total=len(data), desc="Adding nodes"):
    G.add_node(idx, geometry=geom.geometry)

# Step 3: Add edges for overlapping or touching geometries
for i, geom1 in tqdm(data.iterrows(), total=len(data), desc="Adding edges"):
    for j, geom2 in data.iterrows():
        if i != j and (geom1.geometry.overlaps(geom2.geometry) or geom1.geometry.touches(geom2.geometry)):
            G.add_edge(i, j)

# Step 4: Find connected components
connected_components = list(nx.connected_components(G))

# Step 5: Assign group IDs
group_id = 0
data['overlap_group'] = -1
for component in tqdm(connected_components, desc="Assigning group IDs"):
    for idx in component:
        data.at[idx, 'overlap_group'] = group_id
    group_id += 1

data.head()

In [ ]:
# check the number of groups
data.overlap_group.nunique()

In [ ]:
# check the number of tiles in each group by plotting a hist
data.overlap_group.value_counts()